# Ottenimento link news

Questo script si pone come obiettivo quello di recuperare brevi descrizioni dal sito https://footballpredictions.com/ pubblicati prima di ciascun match.
Andiamo a recuperare i dati relativi ad ogni partita andando a cercare nel link il giorno relativo alla partita. 

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
page = 'https://footballpredictions.com/footballpredictions/?date='
format_date = '%d-%m-%Y'

Idee da implementare.
Raggruppamento per data: 
* raggruppare le partite per la data, in modo tale da cercare una sola volta il link relativo alle partite che si svolgono in quella data e non comporre tanti link quante le partite
* la seconda è recuperare tutte le date e poi nell'array fare un select distinct (remove duplicates)

Una volta ottenuti i link andare a cercare nella pagina tutti i link sotto al div relativo alla Serie A

In [3]:
matches = pd.read_csv('matches.csv', index_col=0)[['h_team', 'a_team', 'date']]

In [12]:
match_dates = matches[['date']]

urls = {}
for i, date in match_dates.iterrows():
    page = 'https://footballpredictions.com/footballpredictions/?date='
    string_data=date.iloc[0]
    converted_data = pd.to_datetime(string_data, format='%Y-%m-%d', errors='coerce').date()
    link_data = converted_data.strftime(format_date)
    page += link_data
    urls[str(converted_data)] = page


{'2021-08-21': 'https://footballpredictions.com/footballpredictions/?date=21-08-2021',
 '2021-08-22': 'https://footballpredictions.com/footballpredictions/?date=22-08-2021',
 '2021-08-23': 'https://footballpredictions.com/footballpredictions/?date=23-08-2021',
 '2021-08-27': 'https://footballpredictions.com/footballpredictions/?date=27-08-2021',
 '2021-08-28': 'https://footballpredictions.com/footballpredictions/?date=28-08-2021',
 '2021-08-29': 'https://footballpredictions.com/footballpredictions/?date=29-08-2021',
 '2021-09-11': 'https://footballpredictions.com/footballpredictions/?date=11-09-2021',
 '2021-09-12': 'https://footballpredictions.com/footballpredictions/?date=12-09-2021',
 '2021-09-13': 'https://footballpredictions.com/footballpredictions/?date=13-09-2021',
 '2021-09-17': 'https://footballpredictions.com/footballpredictions/?date=17-09-2021',
 '2021-09-18': 'https://footballpredictions.com/footballpredictions/?date=18-09-2021',
 '2021-09-19': 'https://footballpredictions

Ricerca sulla pagina footballpredictions.com

In [9]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 
                'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

Ho notato che i link relativi alle news contengono la parola serieapredictions, quindi ottengo tutti i link e poi filtro per quelli contenente la parola

Quello che vado a fare è creare un dizionario dove come chiave ho la data del match, mentre come valore ho un array di link che portano a pagine contenenti le news e predizioni delle squadre coinvolte nel match. 

In [10]:
from tqdm import tqdm

links_of_pages_by_date = {}
for key, link_page in tqdm(urls.items()):
    pageTree = requests.get(link_page)
    soup = BeautifulSoup(pageTree.text, features="lxml")
    links = soup.find_all('a', href=True)
    links = [link['href'] for link in links if 'serieapredictions' in link['href']]
    cleaned_links = list(dict.fromkeys(links))
    cleaned_links.pop(0)
    links_of_pages_by_date[key] = cleaned_links

100%|██████████| 119/119 [05:22<00:00,  2.71s/it]


Salvataggio del dizionario su file json. In questo modo la prossima volta lo leggo senza andare a fare scraping per riottenere i link

In [11]:
import json
with open("keyDat_valueLinks.json", "w") as fp:
    json.dump(links_of_pages_by_date,fp, indent=4) 
